In [44]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import xgboost

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt

In [4]:
df = pd.read_csv ('mushrooms.csv')

In [5]:
X = df.iloc[:, 1:].values
y = df.iloc[:,0].values

In [6]:
#Splitting dataset into 75/25 split for training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [7]:
#One-hot encoding the variables in the factors dataset
from sklearn.preprocessing import OneHotEncoder
x_encoder = OneHotEncoder()
x_encoder.fit(X_train)
encX_train = x_encoder.transform(X_train)
encX_test = x_encoder.transform(X_test)

In [8]:
#Employing feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
scX_train = sc.fit_transform(encX_train)
scX_test = sc.transform(encX_test)

In [9]:
#Attempt One (rough draft)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(scX_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=0)

In [10]:
#Results from attempt one
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(scX_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1061    0]
 [   0  970]]


1.0

In [11]:
y_pred

array(['p', 'e', 'e', ..., 'p', 'e', 'e'], dtype=object)

In [12]:
y_test

array(['p', 'e', 'e', ..., 'p', 'e', 'e'], dtype=object)

In [13]:
idTest = list(range(0, len(y_test)))
len(idTest)

2031

In [14]:
dataset = pd.DataFrame({'label': idTest, 'prediction': list(y_pred), 'actual': list(y_test)}, columns=['label', 'prediction', 'actual'])

In [15]:
dataset

,label,prediction,actual
0,0,p,p
1,1,e,e
2,2,e,e
3,3,e,e
4,4,e,e
...,...,...,...
2026,2026,e,e
2027,2027,p,p
2028,2028,p,p
2029,2029,e,e


In [ ]:
#Simple Random forest regression is fine however can we build a more resilient random forest regression here?

In [9]:
#Attempt Two - defining the model
from xgboost import XGBRFClassifier
model = XGBRFClassifier(objective='binary:logistic',
                            n_estimators=100,
                                subsample=0.9,
                                   colsample_bynode=0.7)

In [10]:
#Defining model evaluation procedure of cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
cross_val = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [11]:
#Evaluate model and collect the scores
n_scores = cross_val_score(model, scX_train, y_train, scoring='accuracy', cv=cross_val, n_jobs=-1)

In [12]:
# evaluate xgboost random forest algorithm for classification
from numpy import mean
from numpy import std
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 1.000 (0.000)


In [13]:
#Fitting model
model.fit(scX_train, y_train)

C:\Users\abhi\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:26:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=0.7, colsample_bytree=1,
                enable_categorical=False, gamma=0, gpu_id=-1,
                importance_type=None, interaction_constraints='',
                max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=8,
                num_parallel_tree=100, objective='binary:logistic',
                predictor='auto', random_state=0, reg_alpha=0,
                scale_pos_weight=1, subsample=0.9, tree_method='exact',
                validate_parameters=1, verbosity=None)

In [14]:
y_pred = model.predict(scX_test)

In [15]:
accuracy = float(np.sum(y_pred==y_test))/y_test.shape[0]
print("accuracy of XGBOOSTED trees: %f" %(accuracy))

accuracy of XGBOOSTED trees: 1.000000


In [16]:
idTest = list(range(0, len(y_test)))
newDataset = pd.DataFrame({'label': idTest, 'prediction': list(y_pred), 'actual': list(y_test)}, columns=['label', 'prediction', 'actual'])

In [17]:
newDataset

,label,prediction,actual
0,0,p,p
1,1,e,e
2,2,e,e
3,3,e,e
4,4,e,e
...,...,...,...
2026,2026,e,e
2027,2027,p,p
2028,2028,p,p
2029,2029,e,e
